# Agenda

1. What are decorators?
2. Decorating functions
3. Enclosing functions (and taking advantage of them with decorators)

Decorators in Python ≠ Decorators in design patterns

# Things to remember about functions in Python

1. When we use `def` to define a function, we are actually doing two things:
    - Creating a function object
    - Assigning that function object to a variable
2. Inside of a function, assigning to a variable creates a local variable
3. Functions are objects, which we can return (like any other object) from a function.

In [1]:
# let's say that we have two functions

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())

A!

B!



In [2]:
# our company has now decided, as a matter of policy, that anything we print
# in our programs must have dashed lines above and below it

lines = '-' * 10 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

----------
A!
----------

----------
B!
----------



In [3]:
# this violates the DRY rule -- don't repeat yourself

# how can we DRY up our code, and not repeat ourselves?

lines = '-' * 10 + '\n'

# this function will take another function as an argument, and put its output in lines
def with_lines(func):    
    return f'{lines}{func()}{lines}'


def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))  # I'm not running a or b directly.. with_lines will run them for me
print(with_lines(b))

----------
A!
----------

----------
B!
----------



In [4]:
# the boss is unhappy that we changed the API
# we need to have lines... but we need to have them without changing how we call a and b

# we're going to solve this problem by having with_lines return a *function*, 
# rather than a string

lines = '-' * 10 + '\n'

def with_lines(func):    
    def wrapper():    # define an inner function
        return f'{lines}{func()}{lines}'
    return wrapper    # return the inner function

def a():
    return f'A!\n'
a_with_lines = with_lines(a)


def b():
    return f'B!\n'
b_with_lines = with_lines(b)

print(a_with_lines())
print(b_with_lines())

----------
A!
----------

----------
B!
----------



In [5]:
# let's now make sure that we have the original API working

lines = '-' * 10 + '\n'

def with_lines(func): # func will hold onto our reference to our original function   
    def wrapper():    # define an inner function
        return f'{lines}{func()}{lines}'    # we refer to func in the inner function... so it sticks around
    return wrapper    # return the inner function

def a():
    return f'A!\n'
a = with_lines(a)  # we're redefining a so that it's now the result of calling with_lines(a)


def b():
    return f'B!\n'
b = with_lines(b)

print(a())
print(b())

----------
A!
----------

----------
B!
----------



In [6]:
# let's use Python decorator syntax!

lines = '-' * 10 + '\n'

def with_lines(func):
    def wrapper():   
        return f'{lines}{func()}{lines}'    
    return wrapper  

@with_lines    # this is precisely the same as line 13!
def a():
    return f'A!\n'
# a = with_lines(a) 


@with_lines   # this is precisely the same as line 19
def b():
    return f'B!\n'
# b = with_lines(b)

print(a())
print(b())

----------
A!
----------

----------
B!
----------



# This is a decorator in Python!

- It's a function
- That function takes an argument, another function
- It also returns a function (traditionally called `wrapper`)
- The returned function is then assigned back to our original function's name

### We have three functions here:

- Originally defined one (`a` or `b`)
- Decorator function (`with_lines`)
- Inner function, which replaces our original functions (traditionally called `wrapper`)

# Why would we do this?

This means that we can "hijack" a function at two points in time:

- When it is defined
- When it is run

We can do all sorts of things to it:
- Replace the function with another one at definition time
- Filter the arguments
- Filter the return values
- Cache values across different calls
- Put in logging/timing information
- Add security code

Technically speaking a decorator is:

- A callable (function or class)
- That takes a callable (function or class) as an argument
- And returns a callable (function or class)

In [8]:
# what if our function takes arguments?

lines = '-' * 10 + '\n'

def with_lines(func):
    def wrapper():   
        return f'{lines}{func()}{lines}'    
    return wrapper  

@with_lines    # this is precisely the same as line 13!
def a():
    return f'A!\n'
# a = with_lines(a) 


@with_lines   # this is precisely the same as line 19
def b():
    return f'B!\n'
# b = with_lines(b)

@with_lines
def myadd(first, second):
    return f'{first} + {second} = {first+second}\n'

print(a())
print(b())
print(myadd(10, 3))   # when I call myadd here, I'm not really calling our original myadd, but wrapper

----------
A!
----------

----------
B!
----------



TypeError: with_lines.<locals>.wrapper() takes 0 positional arguments but 2 were given

In [10]:
# what if our function takes arguments?

lines = '-' * 10 + '\n'

def with_lines(func):
    def wrapper(*args):      # take any number of positional arguments
        return f'{lines}{func(*args)}{lines}'      # turn our tuple args into positional arguments
    return wrapper  

@with_lines    # this is precisely the same as line 13!
def a():
    return f'A!\n'
# a = with_lines(a) 


@with_lines   # this is precisely the same as line 19
def b():
    return f'B!\n'
# b = with_lines(b)

@with_lines
def myadd(first, second):
    return f'{first} + {second} = {first+second}\n'

print(a())
print(b())
print(myadd(10, 3))   # when I call myadd here, I'm not really calling our original myadd, but wrapper

----------
A!
----------

----------
B!
----------

----------
10 + 3 = 13
----------



# Exercise: `timefunc`

1. Write two functions, `slow_add` and `slow_mul`, each of which takes two arguments. They return the sum and product of those two arguments, respectively. You should add a `time.sleep` in there, so that they'll be a little slow. You can even use `time.sleep(random.randint(0, 3))` for 0-3 seconds of sleeping.
2. Write a decorator, `timefunc`, which will run the function as per usual -- but will keep track of how long the function took to run, by calling `time.time()` before and after doing so.
3. The timing of the function should be written into a file, `timing.txt`, in the current directory. Each line of `timing.txt` should have the called function's name (which you can get via `__name__` on the function object) and how long it took to run.

```python
print(slow_add(3, 5))
print(slow_mul(10, 2))
print(slow_add(7, 8))
print(slow_mul(2, 4))
```

After running the above code, `timing.txt` should have four lines, each line starting with the function name.

In [13]:
import time
import random

def timefunc(func):          # this is the decorator; it's called once per function, which is the argument
    def wrapper(*args):      # this replaces the decorated function; it's called once per invocation
        start_time = time.time()
        value = func(*args)
        total_time = time.time() - start_time
        
        with open('timing.txt', 'a') as outfile:                # open the file for appending
            outfile.write(f'{func.__name__}\t{total_time}\n')   # add one line: function name + total_time
        
        return value
    return wrapper           # the outer function returns the inner function when it's called

@timefunc
def slow_add(first, second):
    time.sleep(random.randint(0, 3))
    return first + second

@timefunc
def slow_mul(first, second):
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(3, 5))
print(slow_mul(10, 2))
print(slow_add(7, 8))
print(slow_mul(2, 4))

8
20
15
8


In [14]:
# in Jupyter, ! means: run a command in the host OS
# in Unix/Mac, cat means: show me the contents of a file

!cat timing.txt

slow_add	1.0043787956237793
slow_mul	3.004262924194336
slow_add	2.000728130340576
slow_mul	1.0007259845733643


What if I have a function that doesn't have any state, and which always returns the same value, given the same arguments? If that function takes a long time to run, then maybe it would be helpful for me to do some caching.  That is: I'll check to see what arguments I get, and if I've already seen those arguments in the past, then I'll just return the cached value.

The first time that I see a set of arguments, I'll really run the function, and then cache whatever I get back.

This is known as "memoization."  It has been around since about the 1950s.

# Exercise: Memoization

1. Let's reuse our `slow_add` and `slow_mul` functions.
2. Instead of the `timing` decorator, let's use a `memoize` decorator. 
3. `memoize` will look at the arguments that a function gets:
    - If we've seen these arguments before, then we'll return the previous calls' value.
    - If this is the first time we've seen these arguments, then we call the function, and then cache the value we got back.
4. Print some debugging text indicating whether this is the first time or a non-first time the function is called.    

In [17]:
def memoize(func):      # decorator; called once on the decorated function
    cache = {}
    def wrapper(*args): # inner function; called each time we invoke the decorated function
        if args not in cache:
            print('Caching result for args {args}')
            cache[args] = func(*args)
            
        # we can now always return the value
        return cache[args]

    return wrapper

@memoize
def slow_add(first, second):
    print(f'Now really running slow_add with {first} and {second}')
    time.sleep(random.randint(0, 3))
    return first + second

@memoize
def slow_mul(first, second):
    print(f'Now really running slow_mul with {first} and {second}')
    time.sleep(random.randint(0, 3))
    return first * second

print(slow_add(3, 5))
print(slow_mul(3, 2))
print(slow_add(3, 5))
print(slow_mul(10, 2))


Now really running slow_add with 3 and 5
8
Now really running slow_mul with 10 and 2
20
Now really running slow_add with 3 and 5
8
Now really running slow_mul with 10 and 2
20
